<a href="https://colab.research.google.com/github/disneyono/oono/blob/main/shashin_densono_shinpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install janome

from janome.tokenizer import Tokenizer
import zipfile
import os.path, urllib.request as req

# ZIPファイルをダウンロード --- (※1)
url = "http://www.aozora.gr.jp/cards/000042/files/60978_ruby_74893.zip"
local = "60978_ruby_74893.zip"
if not os.path.exists(local):
    print("ZIPファイルをダウンロード")
    req.urlretrieve(url, local)

# ZIPファイル内のテキストファイルを読む --- (※2)
zf = zipfile.ZipFile(local, 'r') # zipファイルを読む
fp= zf.open('shashin_densono_shinpo.txt', 'r') # アーカイブ内のテキストを読む
bindata = fp.read()
txt = bindata.decode('shift_jis') # テキストがShift_JISなのでデコード

# 形態素解析オブジェクトの生成 --- (※3)
t = Tokenizer()

# テキストを一行ずつ処理 --- (※4)
word_dic = {}
lines = txt.split("\r\n")
for line in lines:
    malist = t.tokenize(line)
    for w in malist:
        word = w.surface
        ps = w.part_of_speech # 品詞
        if ps.find('名詞') < 0: continue # 名詞だけカウント --- (※5)
        if not word in word_dic:
            word_dic[word] = 0
        word_dic[word] += 1 # カウント

# よく使われる単語を表示 --- (※6)
keys = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)
for word,cnt in keys[:50]:
    print("{0}({1}) ".format(word,cnt), end="")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
機(12) 写真(9) これ(9) もの(8) の(8) 事(7) 式(7) 円筒(7) 十(7) 膜(6) 年(6) 受信(6) 地(5) コルン(5) よう(5) セレニウム(5) 電流(5) 日(5) 電送(4) 中(4) 一(4) 成(4) 効(4) 筒(4) 廻転(4) 原板(4) 写し(4) センチメートル(4) 月(4) 寺田(3) 寅彦(3) 入力(3) 本(3) 目的(3) 光線(3) 濃淡(3) 発信(3) 板(3) 針(3) ペン(3) 八(3) .(3) -------------------------------------------------------(2) 例(2) フィルム(2) １(2) 字(2) 上げ(2) 針金(2) 発明(2) 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/検索技術/
!pip install gensim
from janome.tokenizer import Tokenizer
from gensim.models import word2vec
import re

# テキストファイルの読み込み --- (※1)
bindata = open('shashin_densono_shinpo.txt', 'rb').read()
text = bindata.decode('shift_jis')

# テキストの先頭にあるヘッダとフッタを削除 --- (※2)
text = re.split(r'\-{5,}',text)[2]
text = re.split(r'底本：', text)[0]
text = text.strip()

# 形態素解析 --- (※3)
t = Tokenizer()
results = []
# テキストを一行ずつ処理する
lines = text.split("\r\n")
for line in lines:
    s = line
    s = s.replace('｜', '')
    s = re.sub(r'《.+?》', '', s) # ルビを削除
    s = re.sub(r'［＃.+?］', '', s) # 入力注を削除
    tokens = t.tokenize(s) # 形態素解析
    # 必要な語句だけを対象とする --- (※4)
    r = []
    for tok in tokens:
        if tok.base_form == "*": # 単語の基本系を採用
            w = tok.surface
        else:
            w = tok.base_form
        ps = tok.part_of_speech # 品詞情報
        hinsi = ps.split(',')[0]
        if hinsi in ['名詞', '形容詞', '動詞', '記号']:
            r.append(w)
    rl = (" ".join(r)).strip()
    results.append(rl)
    #print(rl) # --- 画面に分かち書きした行を表示

# 書き込み先テキストを開く --- (※5)
wakati_file = 'shashin_densono_shinpo.wakati'
with open(wakati_file, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))

# Word2Vecでモデルを作成 --- (※6)
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data, 
    size=200, window=10, hs=1, min_count=2, sg=1)
model.save('shashin_densono_shinpo.model')
print('ok')





Mounted at /content/gdrive
/content/gdrive/MyDrive/検索技術
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ok


In [ ]:
from gensim.models import word2vec


model = word2vec.Word2Vec.load('shashin_densono_shinpo.model')

for word in ['電流']:
    words = model.most_similar(positive=[word])
    n = [w[0] for w in words]
    print(word,"=", ",".join(n))



電流 = 、,膜,。,これ,地,する,発信,送る,受信,つける


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys
